In [1]:
import pandas as pd
import numpy as np
from ortools.sat.python import cp_model
from datetime import datetime, timedelta

In [2]:
import sys
sys.path.append('/home/cayena/Projeto_thefamily/TeacherScheduler/')

In [3]:
from streamlit_app.utils import transform_classes_dateframe,transform_teacher_dataframe,transform_alocation_dataframe,base_selection,expand_rows,replicate_row,clean_data
from streamlit_app.teacher_alocation import TeacherScheduler
from streamlit_app.validador import validador

2025-05-26 08:55:50.647 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-26 08:55:50.651 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-26 08:55:50.655 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-26 08:55:50.659 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [4]:
aulas_raw = pd.read_excel('/home/cayena/Downloads/rota_teste_final.xlsx')

professores_raw = pd.read_excel('/home/cayena/Downloads/prof recentes.xlsx')

In [5]:
df_result =transform_classes_dateframe(aulas_raw)
df_professor = transform_teacher_dataframe(professores_raw)

In [6]:
Ts = TeacherScheduler(df_result, df_professor)
base_alocada = Ts.schedule_teachers(use_soft_constrait=0)
base_alocada

9135
4


,professores_alocados,nome grupo
0,MADDIE,ABU DHABI EAD - 4X
1,MARIA,ANDERSON VIP EAD - 1X
2,TELMA,AUCKLAND P - 4X
3,LYENE,BAKERSFIELD P - 4X
4,MARIA,BELIZE P - 3X
...,...,...
139,BELLA,NORTHAMPTOM P - 3X
140,CARLOS,PHILADELPHIA P - 3X
141,JULIANA,SAN JUAN EAD - 3X
142,SAMARA,SCOTT EAD - 3X


In [7]:
df_final,aulas_nao_alocadas = transform_alocation_dataframe(aulas_raw,base_alocada)
aulas_nao_alocadas

,nome grupo,horario,unidade,dias da semana,stage,livro,modalidade,grupo,n aulas,parag atual grupo,...,rescisao,permuta,bolsista,n_total_alunos,stage_formula,parag_final_formula,num_1_sempre,ultimo_professor,penultimo_professor,restricoes_professor


In [8]:
numero_aulas = df_final.groupby('teacher').agg({'n aulas':'sum'}).sort_values('n aulas',ascending=False).reset_index()
numero_aulas = pd.merge(numero_aulas,df_professor[['TEACHER','MEDIA']],left_on='teacher',right_on='TEACHER',how='left')

In [9]:
numero_aulas

,teacher,n aulas,TEACHER,MEDIA
0,BRUNO MORGILLO,27,BRUNO MORGILLO,24
1,FELIPE,23,FELIPE,20
2,BELLA,22,BELLA,24
3,LEONEL,20,LEONEL,24
4,RAPHAEL,20,RAPHAEL,24
5,MARIA,20,MARIA,24
6,FERNANDO,20,FERNANDO,24
7,BARBARA,19,BARBARA,16
8,DARLENE,17,DARLENE,16
9,MADDIE,16,MADDIE,20


In [18]:
df_professor[df_professor['MEDIA']<5]

,TEACHER,Email,FERIAS,MEDIA,PRESENCIAL,ONLINE,SATÉLITE,JARDIM,VICENTINA,SEGUNDA,...,SOLICITAÇÕES,PREFERENCIAS,GRAVAÇÃO DE AULAS,Solicitações,Retsrições,HORÁRIO,CARRO,UNIDADE,UNIDADE.1,UNIDADE.2
10,DIEGO - ESP,outrogdrive38@gmail.com,NaN,4,0,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,GIOVANNA,giovanna.rsilva14@gmail.com,NaN,4,0,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,LARISSA ANDRADE,larissaandrade.tfidiomas@gmail.com,NaN,4,0,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,LEONARDO,leonardoalegrio@gmail.com,NaN,4,1,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_final[df_final['penultimo_professor'] == df_final['ultimo_professor']]

,nome grupo,horario,unidade,dias da semana,stage,livro,modalidade,grupo,n aulas,parag atual grupo,...,rescisao,permuta,bolsista,n_total_alunos,stage_formula,parag_final_formula,num_1_sempre,ultimo_professor,penultimo_professor,restricoes_professor


In [11]:
df_final[['teacher','ultimo_professor','penultimo_professor']]

,teacher,ultimo_professor,penultimo_professor
0,MADDIE,MADDIE,FERNANDO
1,MARIA,MARIA,BRUNO MORGILLO
2,TELMA,TELMA,DARLENE
3,MADISON,MADISON,CLAUDIO
4,LYENE,LYENE,FELIPE
...,...,...,...
139,DARLENE,DARLENE,BRUNO MORGILLO
140,BELLA,BELLA,CAIO
141,CARLOS,CARLOS,CAIO
142,LYENE,LYENE,MADISON


## soluçoes ##
1 - tentar olhar para o minimo de aulas (botar o minimo para todos) / vai dar errado.
2 - dar um peso grande para quem tem muita aulas (>= 20) e pouco peso para que tem pouca aulas (<=10).
 2.1 - 24 aulas = minimo vai ser 24 - 3 = 21 / 12 aulas = minimo vai ser 12 - 6 = 6 aulas 
3 - pensar em algo para que tente alocar o maximo, o minimo mas se nao der, tudo bem.